# Support Vector Machines

- an ML algorithm, capable of performing linear / non-linear classification, regressions, novelty detection.

### Linear

A linear SVM predicts a class by computing the decision function
$$\mathbf{\theta}^T\mathbf{x} = \theta_0 x_0 + \dots + \theta_n x_n$$
If the result is positive, then the class is positive. And vice-versa.

#### Margin

One of main SVM concepts;  
Apart from trying to fit the data to classify it, the SVM looks for most "balanced" line (if considering linear SVMs).  
* Hard Margin: trying to fit the margin the "stupid" way. Very sensible to outliers.  
* Soft Margin: in fact, lets some of the sample instances breaking the margin. The regularization term with should be considered. Depending on the value of the hyperparameter, margin-breaking conditions become more or less stricter.

The margin linear SVM classifier objective can be expressed as the constrained optimization problem.
> ##### Hard margin linear SVM classifier objective
> minimize ${1\over 2} \mathbf{w^\intercal w}$  
> subject to $t^{(i)}(\mathbf{w^\intercal x^{(i)}} + b)\ge 1$, for $i = 1, 2, \dots, m$

For a soft margin, the slack variable $\zeta^{(i)} \ge 0$ for each instance should be considered.  
It measures how much the instance $i$ is allowed to violate the margin.

> ##### Soft margin linear SVM classifier objective
> minimize ${1\over 2} \mathbf{w^\intercal w} + C \sum_{i = 1}^m \zeta^{(i)}$  
> subject to $t^{(i)}(\mathbf{w^\intercal x^{(i)}} + b)\ge 1 - \zeta^{(i)}$ and $\zeta^{(i)} \ge 0 $, for $i = 1, 2, \dots, m$

In [1]:
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 2)
svm_clf = make_pipeline(StandardScaler(),
 LinearSVC(C=1, random_state=42))
svm_clf.fit(X, y)
X_new = [[5.5, 1.7], [5.0, 1.5]]
print(f"Predictions: {svm_clf.predict(X_new)}, decision_function: {svm_clf.decision_function(X_new)}")


Predictions: [ True False], decision_function: [ 0.66163816 -0.22035761]


### Nonlinear

Sometimes, the dataset is not linearly seperable. In that case, adding more polynomial features can be considered.

Another way is adding similarity features which measure how much each instance resembles a particular landmarl.

More advanced approach is using kernel trick which allows to consider more features without adding complexity.

### SVM Regression

Instead of finding the largest margin, SVM tries to fit as many instances as possible without violating the margin rule.

### The Dual Problem

Given a constrained optimization problem, known as the primal problem, it is possible to express a different but closely related problem, called its dual problem. The solution to the dual problem typically gives a lower bound to the solution of the primal problem, but under some conditions it can have the same solution as the  primal problem.  
*Fortunately, the SVM problem meets these conditions.*

> ##### Dual form of the linear SVM objective
> $$\text{minimize}_{\alpha} \quad \frac{1}{2} \sum_{i=1}^m \sum_{j=1}^m \alpha^{(i)} \alpha^{(j)} t^{(i)} t {(j)} \mathbf{x}^{(i)} \cdot \mathbf{x}^{(j)} - \sum_{i=1}^m \alpha^{(i)} $$
> $$ \text{subject to} \quad \alpha^{(i)} \geq 0 \quad \text{for all } i = 1, 2, \dots, m, \quad \text{and} \quad \sum_{i=1}^m \alpha^{(i)} t^{(i)} = 0 $$

Once the vector $\hat{\alpha}$ is found, $\hat{\mathbf{w}}$ and $\hat{b}$ can be computed.

> ##### From the dual solution to the primal solution
>$$\mathbf{w} = \sum_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} \mathbf{x}^{(i)}$$
>$$\hat{b} = \frac{1}{n_s} \sum_{i=1}^m \left( t^{(i)} - \mathbf{w} \cdot \mathbf{x}^{(i)} \right) \quad \text{where} \quad \hat{\alpha}^{(i)} > 0 $$

#### Kernelized SVMs

Suppose you want to apply a second-degree polynomial transformation to a twodimensional training set 

> #####  Second-degree polynomial mapping
> $$\varphi(\mathbf{x}) = \varphi \left( 
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \right) = 
\begin{bmatrix} 
x_1^2 \\ 
\sqrt{2} x_1 x_2 \\ 
x_2^2 
\end{bmatrix}$$

But, considering the dual problem, the following can be achieved.

> ##### Kernel trick for a second-degree polynomial mapping
> $$\varphi(\mathbf{a})^\top \varphi(\mathbf{b}) = 
\begin{bmatrix}
a_1^2 \\
\sqrt{2} a_1 a_2 \\
a_2^2
\end{bmatrix}^\top
\begin{bmatrix}
b_1^2 \\
\sqrt{2} b_1 b_2 \\
b_2^2
\end{bmatrix} 
= a_1^2 b_1^2 + 2 a_1 b_1 a_2 b_2 + a_2^2 b_2^2 $$
> $$
= (a_1 b_1 + a_2 b_2)^2 = 
\left( \begin{bmatrix} a_1 \\ a_2 \end{bmatrix}^\top 
\begin{bmatrix} b_1 \\ b_2 \end{bmatrix} \right)^2 
= (\mathbf{a}^\top \mathbf{b})^2$$

The result will be the same, but the second option is way less computationally expensive.

So, **Kernel** is a function capable of computing the dot product $\varphi(\mathbf{a})^\intercal \varphi(\mathbf{b})$, based only on the original vectors a and b, without having to compute the transformation $\varphi$.

> ##### Common kernels
> Linear:     $K(\mathbf{a}, \mathbf{b}) = \mathbf{a}^\top \mathbf{b} $  
> Polynomial:   $K(\mathbf{a}, \mathbf{b}) = (\gamma \mathbf{a}^\top \mathbf{b} + r)^d$  
> Gaussian RBF:  $K(\mathbf{a}, \mathbf{b}) = \exp\left(-\gamma \|\mathbf{a} - \mathbf{b}\|^2\right)$  
> Sigmoid:    $K(\mathbf{a}, \mathbf{b}) = \tanh\left(\gamma \mathbf{a}^\top \mathbf{b} + r\right)$  


> ##### Making predictions with a kernelized SVM
> $$
h_{\mathbf{w}, \hat{b}}(\varphi(\mathbf{x}^{(n)})) = 
\mathbf{\hat{w}}^\top \varphi(\mathbf{x}^{(n)}) + \hat{b}
= \left( \sum_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} \varphi(\mathbf{x}^{(i)}) \right)^\top \varphi(\mathbf{x}^{(n)}) + \hat{b} $$
> $$
= \sum_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} \left( \varphi(\mathbf{x}^{(i)})^\top \varphi(\mathbf{x}^{(n)}) \right) + \hat{b}$$
> $$
= \sum_{i=1}^m \hat{\alpha}^{(i)} t^{(i)} K(\mathbf{x}^{(i)}, \mathbf{x}^{(n)}) + \hat{b}
\quad \text{where} \quad \hat{\alpha}^{(i)} > 0$$

> ##### Using the kernel trick to compute the bias term
> $$ \hat{b} = \frac{1}{n_s} \sum_{i=1 \atop \hat{\alpha}^{(i)} > 0}^m \left(t^{(i)} - \hat{\mathbf{w}}^T\varphi(\mathbf{x}^{(i)})\right) = \frac{1}{n_s} \sum_{i=1 \atop \hat{\alpha}^{(i)} > 0}^m \left(t^{(i)} - \left(\sum_{j=1}^m \hat{\alpha}^{(j)} t^{(j)} \varphi(\mathbf{x}^{(j)})\right)^T \varphi(\mathbf{x}^{(i)})\right) $$
> $$= \frac{1}{n_s} \sum_{i=1 \atop \hat{\alpha}^{(i)} > 0}^m \left(t^{(i)} - \sum_{j=1 \atop \hat{\alpha}^{(j)} > 0}^m \hat{\alpha}^{(j)} t^{(j)} K(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})\right) $$